In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset

In [10]:
dataset = load_dataset("json", data_files={"train": "nerData-train.json", "test": "nerData-test.json"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/studio-lab-user/.cache/huggingface/datasets/json/default-0ac1243306f124ef/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset["train"][0]

{'ner_entities': ['B-PRON',
  'O',
  'B-PRON',
  'B-VERB',
  'O',
  'B-NOUN',
  'B-TIME',
  'I-TIME',
  'O'],
 'sentence': 'What did I have for breakfast this morning?',
 'ner_tags': [40, 0, 40, 37, 0, 38, 13, 31, 0],
 'tokens': ['What',
  'did',
  'I',
  'have',
  'for',
  'breakfast',
  'this',
  'morning',
  '?']}

In [5]:
len(dataset["test"])

757

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [11]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
tokenized_set = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3563 [00:00<?, ? examples/s]

Map:   0%|          | 0/891 [00:00<?, ? examples/s]

In [14]:
tokenized_set.keys()

dict_keys(['train', 'test'])

In [15]:
tokenized_set["train"][0]

{'ner_entities': ['B-PRON',
  'O',
  'B-PRON',
  'B-VERB',
  'O',
  'B-NOUN',
  'B-TIME',
  'I-TIME',
  'O'],
 'sentence': 'What did I have for breakfast this morning?',
 'ner_tags': [40, 0, 40, 37, 0, 38, 13, 31, 0],
 'tokens': ['What',
  'did',
  'I',
  'have',
  'for',
  'breakfast',
  'this',
  'morning',
  '?'],
 'input_ids': [101, 2054, 2106, 1045, 2031, 2005, 6350, 2023, 2851, 1029, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 40, 0, 40, 37, 0, 38, 13, 31, 0, -100]}

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

2023-06-08 04:41:42.693679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 04:41:43.282461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 04:41:43.284187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 04:41:45.994538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
import evaluate

seqeval = evaluate.load("seqeval")

In [20]:
import numpy as np

label_list = ["O", 
"B-PERSON",
"B-NORP",
"B-FAC",
"B-ORG",
"B-GPE",
"B-LOC",
"B-PRODUCT",
"B-EVENT",
"B-WORK_OF_ART",
"B-LAW",
"B-LANGUAGE",
"B-DATE",
"B-TIME",
"B-PERCENT",
"B-MONEY",
"B-QUANTITY",
"B-ORDINAL",
"B-CARDINAL",
"I-PERSON",
"I-NORP",
"I-FAC",
"I-ORG",
"I-GPE",
"I-LOC",
"I-PRODUCT",
"I-EVENT",
"I-WORK_OF_ART",
"I-LAW",
"I-LANGUAGE",
"I-DATE",
"I-TIME",
"I-PERCENT",
"I-MONEY",
"I-QUANTITY",
"I-ORDINAL",
"I-CARDINAL",
"B-VERB",
"B-NOUN",
"B-SCONJ",
"B-PRON"]


# labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [27]:
id2label = {
    0:"O", 
1:"B-PERSON",
2:"B-NORP",
3:"B-FAC",
4:"B-ORG",
5:"B-GPE",
6:"B-LOC",
7:"B-PRODUCT",
8:"B-EVENT",
9:"B-WORK_OF_ART",
10:"B-LAW",
11:"B-LANGUAGE",
12:"B-DATE",
13:"B-TIME",
14:"B-PERCENT",
15:"B-MONEY",
16:"B-QUANTITY",
17:"B-ORDINAL",
18:"B-CARDINAL",
19:"I-PERSON",
20:"I-NORP",
21:"I-FAC",
22:"I-ORG",
23:"I-GPE",
24:"I-LOC",
25:"I-PRODUCT",
26:"I-EVENT",
27:"I-WORK_OF_ART",
28:"I-LAW",
29:"I-LANGUAGE",
30:"I-DATE",
31:"I-TIME",
32:"I-PERCENT",
33:"I-MONEY",
34:"I-QUANTITY",
35:"I-ORDINAL",
36:"I-CARDINAL",
37:"B-VERB",
38:"B-NOUN",
39:"B-SCONJ",
40:"B-PRON"}

In [28]:
label2id={
"O":0, # ! not classified        
"B-PERSON": 1, # People, including fictional.
"B-NORP": 2,  # Nationalities, religious or political groups.
"B-FAC": 3, # Buildings, airports, highways, bridges, etc.
"B-ORG": 4, # Companies, agencies, institutions, etc.
"B-GPE": 5, # Countries, cities, states.
"B-LOC": 6, # Non-GPE locations, mountain ranges, bodies of water.
"B-PRODUCT": 7, # Objects, vehicles, foods, etc. (Not services.)
"B-EVENT": 8, # Named hurricanes, battles, wars, sports events, etc.
"B-WORK_OF_ART": 9, # Titles of books, songs, etc.
"B-LAW": 10, # Named documents made into laws.
"B-LANGUAGE": 11, # Any named language.
"B-DATE": 12, # Absolute or relative dates or periods.
"B-TIME": 13, # Times smaller than a day.
"B-PERCENT": 14, # Percentage, including "%".
"B-MONEY": 15, # Monetary values, including unit.
"B-QUANTITY": 16, # Measurements, as of weight or distance.
"B-ORDINAL": 17, # "first", "second", etc.
"B-CARDINAL": 18, # Numerals that do not fall under another type.
"I-PERSON": 19, # People, including fictional.
"I-NORP": 20,  # Nationalities, religious or political groups.
"I-FAC": 21, # Buildings, airports, highways, bridges, etc.
"I-ORG": 22, # Companies, agencies, institutions, etc.
"I-GPE": 23, # Countries, cities, states.
"I-LOC": 24, # Non-GPE locations, mountain ranges, bodies of water.
"I-PRODUCT": 25, # Objects, vehicles, foods, etc. (Not services.)
"I-EVENT": 26, # Named hurricanes, battles, wars, sports events, etc.
"I-WORK_OF_ART": 27, # Titles of books, songs, etc.
"I-LAW": 28, # Named documents made into laws.
"I-LANGUAGE": 29, # Any named language.
"I-DATE": 30, # Absolute or relative dates or periods.
"I-TIME": 31, # Times smaller than a day.
"I-PERCENT": 32, # Percentage, including "%".
"I-MONEY": 33, # Monetary values, including unit.
"I-QUANTITY": 34, # Measurements, as of weight or distance.
"I-ORDINAL": 35, # "first", "second", etc.
"I-CARDINAL": 36, # Numerals that do not fall under another type.
"B-VERB": 37, # ! word is verb
"B-NOUN": 38, # ! word is noun or proper noun
"B-SCONJ": 39, # ! word is Subordinating conjunction
"B-PRON": 40 # ! word is Pronoun
}


In [29]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
#len(label2id.keys())
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(label2id.keys()), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [30]:
training_args = TrainingArguments(
    output_dir="my_test_big_ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_set["train"],
    eval_dataset=tokenized_set["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...


Cloning https://huggingface.co/tro9999/my_test_big_ner_model into local empty directory.


To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.253998,0.913921,0.930807,0.922287,0.945884
2,No log,0.214182,0.921346,0.934201,0.927729,0.948764


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=446, training_loss=0.32687757688787483, metrics={'train_runtime': 992.1226, 'train_samples_per_second': 7.183, 'train_steps_per_second': 0.45, 'total_flos': 35351519014734.0, 'train_loss': 0.32687757688787483, 'epoch': 2.0})

In [31]:
text = "The CEO of Microsoft, Satya Nadella, announced yesterday that the company will be releasing a new product next month."

In [32]:
from transformers import pipeline

classifier = pipeline("ner", model="./my_test_big_ner_model")
classifier(text)

[{'entity': 'B-NOUN',
  'score': 0.88468677,
  'index': 2,
  'word': 'ceo',
  'start': 4,
  'end': 7},
 {'entity': 'B-NORP',
  'score': 0.1623697,
  'index': 4,
  'word': 'microsoft',
  'start': 11,
  'end': 20},
 {'entity': 'B-NOUN',
  'score': 0.13884565,
  'index': 7,
  'word': '##ya',
  'start': 25,
  'end': 27},
 {'entity': 'B-NOUN',
  'score': 0.20582804,
  'index': 8,
  'word': 'nad',
  'start': 28,
  'end': 31},
 {'entity': 'B-NOUN',
  'score': 0.13946776,
  'index': 9,
  'word': '##ella',
  'start': 31,
  'end': 35},
 {'entity': 'B-VERB',
  'score': 0.83986384,
  'index': 11,
  'word': 'announced',
  'start': 37,
  'end': 46},
 {'entity': 'B-DATE',
  'score': 0.9633989,
  'index': 12,
  'word': 'yesterday',
  'start': 47,
  'end': 56},
 {'entity': 'B-NOUN',
  'score': 0.86388886,
  'index': 15,
  'word': 'company',
  'start': 66,
  'end': 73},
 {'entity': 'B-VERB',
  'score': 0.9721786,
  'index': 18,
  'word': 'releasing',
  'start': 82,
  'end': 91},
 {'entity': 'B-NOUN',
  